# Demo for cluster analyzing LEEM stacks

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from agfalta.leem.base import LEEMStack
from agfalta.leem.driftnorm import normalize_stack, align_stack
import agfalta.leem.cluster as cluster
import agfalta.leem.plotting as plotting

Normalization and drift correction (if it exists, the corrected stack is loaded directly):

In [2]:
load_prealigned = True

if load_prealigned:
    # either load a pre-aligned image stack like this (then you need to add the energy vector for non-lstk-stacks)
    stack_raw = LEEMStack("media/test_stack_IV_RuO2_normed_aligned.tif")
    stack_raw.energy = np.linspace(3.0, 50.0, len(stack_raw))
else:
    # or you load a raw data set and normalize and align it
    stack_raw = LEEMStack("test_stack_IV_g-Cu")

    stack_raw = normalize_stack(stack_raw, "channelplate.dat", dark_counts=110)
    stack_raw = align_stack(stack_raw, mask_outer=0.2, trafo="translation")

    plt.plot([m[0, 2] for m in stack.alignment])
    plt.plot([m[1, 2] for m in stack.alignment])

stack = stack_raw.copy()

# you may want to delete some frames:
del stack[160]
stack = stack[10:]

FileNotFoundError: 'media/test_stack_IV_RuO2_normed_aligned.tif' does not exist, cannot be read successfully or contains no *.dat files

Stack inspection:

In [ ]:
plotting.plot_movie(stack[::50], fields=(None, None, None, None))

For the principal component analysis, the stack has to be reshaped first with `stack2vectors()`. This cuts away the outer parts of the images and changes the 3-dimensional stack data into a list of 1-dimensional spectra. The new height and width are needed later for transforming back into images.

Then, `pendryfy()` applies the Pendry-R-Factor metric. (The spectra are transformed into the "Y" in Pendry's paper)

`component_analysis()` does what it says and puts out 2 functions and a model. The functions are used to transform between "real spectrum space" and "principal component space". The model holds all information of the analysis. It can be saved via `save_model(model, filename)` and loaded via `trafo, inv_trafo, model = load_pca_model(filename)`

In [ ]:
n_components = 7

X, h, w = cluster.stack2vectors(stack, mask_outer=0.2) # cut away 20% on every side

X = cluster.pendryfy(X, stack.energy)

trafo, inv_trafo, model = cluster.component_analysis(X, "pca", n_components=n_components)
W = trafo(X)

The resulting `W`-vectorlist has the same format as `X`, but in "PCA-space" and with fewer images. It can be converted into a normal stack and then plotted like this:

In [ ]:
comps = cluster.vectors2stack(W, h, w)
plotting.plot_movie(comps, fields=(None, None, None, None))

The actual cluster analysis is simple. The `cluster_analysis` function takes different keyword arguments depending on the algorithm and always returns `labels` and a model (can be saved by `save_model(model, filename)` and loaded by `labels, model = load_cluster_model(filename)`).

`labels` has the same format as `X`, but only one image that contains integers (the "cluster number"). `labels` will be ordered in cluster size, so the smallest cluster has label `0` and the next one `1` and so on.

In [ ]:
n_clusters = 8
labels, model = cluster.cluster_analysis(W, "pc-xmeans", n_clusters=n_clusters, metric="euclidean_square")

To plot the results, the initial stack and its mask is needed to extract the correct IV curves. The labels need to be reshaped correctly.

Label grouping:

* 0: small disordered things and border of 101
* 1: 101-island
* 2: substrate, 3: substrate border, 4: substrate border and 100
* 5: 110 border, 6: 110 parts, 7: 110

In [ ]:
_, axes = plt.subplots(1, 4, figsize=(20, 8))
cmap = "brg"

cluster.plot_clustermap(labels.reshape(h, w), ax=axes[0], cmap=cmap)
axes[0].set_title("Original clusters")

combined_labels = cluster.combine_labels(labels, [(2, 3), (5, 6, 7)])
cluster.plot_clustermap(combined_labels.reshape(h, w), ax=axes[1], cmap=cmap)
axes[1].set_title("Combined labels")

i = 3
single_map = np.zeros_like(labels)
single_map[labels == i] = 1
cluster.plot_clustermap(single_map.reshape(h, w), ax=axes[2], cmap=cmap)
axes[2].set_title(f"Single cluster: {i}")

plotting.plot_img(stack[80], ax=axes[3])
axes[3].set_title("Raw image")

cluster.plot_IVs(stack_raw, combined_labels, mask_outer=0.2, cmap=cmap)  # need the mask that was applied before PCA